## Segmentação dos dados

As características normalmente são extraídas sobre pequenos segmentos de tamanho fixo dos dados, não no dado como um todo. Estes pequenos segmentos são chamados de janelas. A técnica de separar os dados em janelas, recebe o nome de janela deslizante (*sliding window*) e é uma forma de segmentação de dados. Uma boa prática, é definir um passo para essa janela de forma que haja uma sopreposição de dados, para que informações da lacuna de uma janela e outra não sejam perdidas. Na imagem a seguir podemos observar um sinal EOG. Abaixo dele a representação de janelas: *W1*, *W2*, *W3*... Repare que entre as janelas, há uma sobreposição de tamanho *T*. 

Na base de dados que iremos implementar, cada ensaio tem duração de 5 segundos. Se utilizadas janelas com tamanho de 250 ms, resultará em 20 janelas. Ao aplicar uma sobreposição de ~128ms, ficamos com 41 janelas de ~122ms.

Algumas características de EMG trabalham com o dado no domínio da frequência. Quando tais características são aplicadas, é necessário tranformar o dado para o domínio da frequência, utilizando o método de transformação de domínio `STFT`, do inglês *Short-time Fourier transform* (Transformada de Fourier de curto termo). O código divide o dado em segmentos, tanto no domínio do tempo quanto no domínio da frequência.

In [1]:
# Criando dados fictícios

import numpy as np

shape = (60, 4, 1000)
data = np.random.normal(loc=0, scale=10, size=shape).astype(np.float32)
data.shape, data

((60, 4, 1000),
 array([[[ 11.508245  , -13.739167  ,   6.941915  , ...,  23.426107  ,
            4.296021  ,  -5.3623176 ],
         [  6.769362  ,  22.942202  ,   0.38446867, ...,   6.3696885 ,
           -1.2877166 ,  -7.1020417 ],
         [ 10.663297  ,   5.4280615 ,   1.4339345 , ...,  12.7422495 ,
          -11.26237   ,  -2.7682688 ],
         [-21.912968  ,  -7.0221024 ,   5.093717  , ...,  20.832804  ,
            5.738126  ,  11.265911  ]],
 
        [[ -7.853088  ,  -1.9523042 ,   6.8395925 , ...,  26.430939  ,
          -18.71119   ,  13.286494  ],
         [-14.382093  ,   0.92874885,  19.190777  , ...,   5.9452696 ,
           -2.5280483 ,  -1.4467391 ],
         [  7.2778344 ,   3.7659206 , -15.509776  , ...,   6.848239  ,
           -4.067806  ,   4.631517  ],
         [-10.477037  , -10.675446  ,  -2.934232  , ...,   3.820731  ,
           13.32923   ,   0.8785722 ]],
 
        [[ -6.6344194 ,  13.651512  ,  -3.7038217 , ...,  11.2400055 ,
          -11.630405  ,   7

In [2]:
from scipy.signal import stft

step = 29
segment = 64
print('', data.shape)

n_win = int((data.shape[1] - segment) / step) + 1  # Adjusted for the second dimension
ids = np.arange(n_win) * step

# janelas do dado no domínio do tempo
chunks_time = np.array([data[:, k:(k + segment), :].transpose(0, 2, 1) for k in ids])

# janelas do dado no domínio da frequência
_, _, chunks_freq = stft(data, fs=200, nperseg=segment, noverlap=step-1)  # Adjusted noverlap
chunks_freq = chunks_freq.transpose(0, 2, 3, 1)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Domínio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
print(f'Domínio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')

 (60, 4, 1000)
Formato (shape) dos dados depois da divisão de janelas
Domínio do tempo: (0,) - (classes+ensaios, canais, janelas, linhas)
Domínio da frequência:  (60, 33, 29, 4) - (classes+ensaios, canais, janelas, linhas)


*Desafio*: Implementar um algoritmo para encontrar a mesma quantidade de janelas em ambos os domínios (tempo e frequência).